### Define the FormTool class and related classes

The following cell define the FormTool class, which incapsulates a lot of the logics og the system

In [1]:
import json
import operator
from abc import ABC, abstractmethod
from enum import Enum
from typing import (Annotated, Any, Dict, Optional, Type, TypedDict,
                    Union)

from langchain.callbacks.manager import CallbackManagerForToolRun
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.tools import BaseTool, StructuredTool, ToolException
from pydantic import BaseModel, Field, ValidationError, create_model


class FormToolState(Enum):
    INACTIVE = "INACTIVE"
    ACTIVE = "ACTIVE"
    FILLED = "FILLED"

# We cannot pass directly the BaseModel class as args_schema as pydantic will raise errors,
# so we need to create a dummy class that inherits from BaseModel.
class FormToolInactivePayload(BaseModel):
    pass


class FormToolConfirmPayload(BaseModel):
    confirm: bool = Field(
        description="True if the user confirms the form, False if not or wants to change something."
    )


class FormToolOutcome(BaseModel):
    """
    Represents a form tool output.
    The output is returned as str.
    Any other kwarg is returned in the state_update dict
    """

    output: str
    state_update: Optional[Dict[str, Any]] = None
    return_direct: Optional[bool] = False

    def __init__(
        self,
        output: str,
        return_direct: bool = False,
        **kwargs
    ):
        super().__init__(
            output=output,
            return_direct=return_direct
        )
        self.state_update = kwargs


def make_optional_model(original_model: BaseModel) -> BaseModel:
    """
    Takes a Pydantic model and returns a new model with all attributes optional.
    """
    optional_attributes = {
        attr_name: (
            Union[None, attr_type],
            Field(
                default=None, description=original_model.model_fields[attr_name].description)
        )
        for attr_name, attr_type in original_model.__annotations__.items()
    }

    # Define a custom Pydantic model with optional attributes
    new_class_name = original_model.__name__ + 'Optional'
    OptionalModel = create_model(
        new_class_name,
        **optional_attributes,
        __base__=original_model
    )
    OptionalModel.model_config["validate_assignment"] = True

    return OptionalModel


class FormTool(StructuredTool, ABC):
    form: BaseModel = None
    state: Union[FormToolState | None] = None
    skip_confirm: Optional[bool] = False

    # Backup attributes for handling changes in the state
    args_schema_: Optional[Type[BaseModel]] = None
    description_: Optional[str] = None
    name_: Optional[str] = None

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.args_schema_ = None
        self.name_ = None
        self.description_ = None
        self.init_state()

    def init_state(self):
        state_initializer = {
            None: self.enter_inactive_state,
            FormToolState.INACTIVE: self.enter_inactive_state,
            FormToolState.ACTIVE: self.enter_active_state,
            FormToolState.FILLED: self.enter_filled_state
        }
        state_initializer[self.state]()

    def enter_inactive_state(self):
        # Guard so that we don't overwrite the original args_schema if
        # set_inactive_state is called multiple times
        if not self.state == FormToolState.INACTIVE:
            self.state = FormToolState.INACTIVE
            self.name_ = self.name
            self.name = f"{self.name_}Start"
            self.description_ = self.description
            self.description = f"Starts the form {self.name}, which {self.description_}"
            self.args_schema_ = self.args_schema
            self.args_schema = FormToolInactivePayload

    def enter_active_state(self):
        # if not self.state == FormToolState.ACTIVE:
        self.state = FormToolState.ACTIVE
        self.name = f"{self.name_}Update"
        self.description = f"Updates data for form {self.name}, which {self.description_}"
        self.args_schema = make_optional_model(self.args_schema_)
        if not self.form:
            self.form = self.args_schema()
        elif isinstance(self.form, str):
            self.form = self.args_schema(**json.loads(self.form))

    def enter_filled_state(self):
        self.state = FormToolState.FILLED
        self.name = f"{self.name_}Finalize"
        self.description = f"Finalizes form {self.name}, which {self.description_}"
        self.args_schema = make_optional_model(self.args_schema_)
        if not self.form:
            self.form = self.args_schema()
        elif isinstance(self.form, str):
            self.form = self.args_schema(**json.loads(self.form))
        self.args_schema = FormToolConfirmPayload

    def activate(
        self,
        *args,
        run_manager: Optional[CallbackManagerForToolRun] = None,
        **kwargs
    ) -> FormToolOutcome:
        self.enter_active_state()
        return FormToolOutcome(
            output=f"Starting form {self.name}. If the user as already provided some information, call {self.name}.",
            active_form_tool=self,
            tool_choice=self.name
        )

    def update(
        self,
        *args,
        run_manager: Optional[CallbackManagerForToolRun] = None,
        **kwargs
    ) -> FormToolOutcome:
        self._update_form(**kwargs)
        if self.is_form_filled():
            self.enter_filled_state()
            if self.skip_confirm:
                return self.finalize(confirm=True)
            else:
                return FormToolOutcome(
                    active_form_tool=self,
                    output="Form is filled. Ask the user to confirm the information."
                )
        else:
            return FormToolOutcome(
                active_form_tool=self,
                output="Form updated with the provided information. Ask the user for the next field."
            )

    def finalize(
        self,
        *args,
        run_manager: Optional[CallbackManagerForToolRun] = None,
        **kwargs
    ) -> FormToolOutcome:
        if kwargs.get("confirm"):
            # The FormTool could use self.form to get the data, but we pass it as kwargs to 
            # keep the signature consistent with _run
            result = self._run_when_complete(**self.form.model_dump())
            return FormToolOutcome(
                active_form_tool=None,
                output=result,
                return_direct=self.return_direct
            )
        else:
            self.enter_active_state()
            return FormToolOutcome(
                active_form_tool=self,
                output="Ask the user to update the form."
            )

    def _run(
        self,
        *args,
        run_manager: Optional[CallbackManagerForToolRun] = None,
        **kwargs
    ) -> str:
        match self.state:
            case FormToolState.INACTIVE:
                return self.activate(*args, **kwargs, run_manager=run_manager)

            case FormToolState.ACTIVE:
                return self.update(*args, **kwargs, run_manager=run_manager)

            case FormToolState.FILLED:
                return self.finalize(*args, **kwargs, run_manager=run_manager)

    @abstractmethod
    def _run_when_complete(self) -> str:
        """
        Should raise an exception if something goes wrong.
        The message should describe the error and will be sent back to the agent to try to fix it.
        """

    def _update_form(self, **kwargs):
        try:
            model_class = type(self.form)
            data = self.form.model_dump()
            data.update(kwargs)
            # Recreate the model with the new data merged to the old one
            # This allows to validate multiple fields at once
            self.form = model_class(**data)
        except ValidationError as e:
            raise ToolException(str(e))

    def get_next_field_to_collect(
        self,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """
        The default implementation returns the first field that is not set.
        """
        if self.state == FormToolState.FILLED:
            return None

        for field_name, field_info in self.args_schema.__fields__.items():
            if not getattr(self.form, field_name):
                return field_name

    def is_form_filled(self) -> bool:
        return self.get_next_field_to_collect() is None

    def get_tool_start_message(self, input: dict) -> str:
        message = ""
        match self.state:
            case FormToolState.INACTIVE:
                message = f"Starting {self.name}"
            case FormToolState.ACTIVE:
                message = f"Updating form for {self.name}"
            case FormToolState.FILLED:
                message = f"Completed {self.name}"
        return message

class AgentState(TypedDict):
    # The input string
    input: str
    # The list of previous messages in the conversation
    chat_history: Annotated[Optional[list[BaseMessage]], operator.setitem]
    # The outcome of a given call to the agent
    # Needs `None` as a valid type, since this is what this will start as
    agent_outcome: Annotated[Optional[Union[AgentAction,
                                            AgentFinish, None]], operator.setitem]
    # The outcome of a given call to a tool
    # Needs `None` as a valid type, since this is what this will start as
    tool_outcome: Annotated[Optional[Union[FormToolOutcome,
                                           str, None]], operator.setitem]
    # List of actions and corresponding observations
    # Here we annotate this with `operator.add` to indicate that operations to
    # this state should be ADDED to the existing values (not overwrite it)
    intermediate_steps: Annotated[Optional[list[tuple[AgentAction,
                                                      FunctionMessage]]], operator.add]
    error: Annotated[Optional[str], operator.setitem]

    active_form_tool: Annotated[Optional[FormTool], operator.setitem]

    # Used to force the agent to call a specific tool
    tool_choice: Annotated[Optional[str], operator.setitem]


class FormReset(BaseTool):
    name = "FormReset"
    description = """Call this tool when the user doesn't want to complete the form anymore. DON'T call it when he wants to change some data."""
    args_schema: Type[BaseModel] = FormToolInactivePayload

    def _run(self, *args: Any, **kwargs: Any) -> Any:
        return FormToolOutcome(
            active_form_tool=None,
            output="Form reset. Form cleared. Ask the user what he wants to do next."
        )


### Define the model factory

A different model (with different prompts) is used based on the state of the system

In [2]:
import logging
import os
import pprint
import re
from datetime import datetime
from textwrap import dedent

from langchain.agents import create_openai_tools_agent
from langchain.tools import BaseTool
from langchain_core.language_models.chat_models import *
from langchain_core.prompts.chat import (ChatPromptTemplate,
                                         HumanMessagePromptTemplate,
                                         MessagesPlaceholder,
                                         SystemMessagePromptTemplate)
from langchain_core.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI

logger = logging.getLogger(__name__)
pp = pprint.PrettyPrinter(indent=4)

LLM_MODEL = os.environ.get("LLM_MODEL", "gpt-3.5-turbo-0125")

BASE_SYSTEM_MESSAGE_PROMPT = dedent(f"""
    You are a personal assistant trying to help the user. You always answer in English. The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}.
    Don't use any of your knowledge or information about the state of the world. If you need something, ask the user for it or use a tool to find or compute it.
""").strip()
BASE_SYSTEM_MESSAGE_PROMPT_TEMPLATE = SystemMessagePromptTemplate.from_template(
    BASE_SYSTEM_MESSAGE_PROMPT)

PROMPT_FOOTER_MESSAGES = [
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    HumanMessagePromptTemplate(prompt=PromptTemplate(
        template="{input}", input_variables=["input"])),
    MessagesPlaceholder(variable_name="agent_scratchpad")
]

ERROR_CORRECTION_PROMPT = dedent(f"""
    There was an error with your last action.
    Please fix it and try again.

    Error:
    {{error}}.
""").strip()

ERROR_CORRECTION_SYSTEM_MESSAGE = SystemMessagePromptTemplate.from_template(
    ERROR_CORRECTION_PROMPT)

ERROR_CORRECTION_PROMPT_TEMPLATE = ChatPromptTemplate(messages=[
    BASE_SYSTEM_MESSAGE_PROMPT_TEMPLATE,
    ERROR_CORRECTION_SYSTEM_MESSAGE,
    *PROMPT_FOOTER_MESSAGES
])

DEFAULT_PROMPT_TEMPLATE = ChatPromptTemplate(messages=[
    BASE_SYSTEM_MESSAGE_PROMPT_TEMPLATE,
    *PROMPT_FOOTER_MESSAGES
])


def information_to_collect_prompt_template(
    form_tool: BaseTool,
    information_to_collect: str
):
    return SystemMessagePromptTemplate.from_template(dedent(
        f"""
        Help the user fill data for {form_tool.name}. Ask to provide the needed information.
        Now you must should update the form with any information the user provided or ask the user to provide a value for the field "{information_to_collect}".
        You MUST use the {form_tool.name} tool to update the stored data each time the user provides one or more values.
        """
    ).strip())


def ask_for_confirmation_prompt_template(
    form_tool: BaseTool
):    
    information_collected = re.sub("}", "}}", re.sub("{", "{{", str(
    {name: value for name, value in form_tool.form.__dict__.items() if value})))

    return SystemMessagePromptTemplate.from_template(dedent(
        f"""
        Help the user fill data for {form_tool.name}. You have all the information you need.
        Show the user all of the information using bullet points and ask for confirmation:
        {information_collected}
        If he agrees, call the {form_tool.name} tool one more time with confirm=True.
        If he doesn't or want to change something, call it with confirm=False.
        """
    ).strip())


class ModelFactory:

    @staticmethod
    def build_model(
        state: AgentState,
        tools: List[BaseTool] = []
    ):
        builder = ModelFactory.build_default_model
        if state.get("error"):
            builder = ModelFactory.build_error_model
        elif state.get("active_form_tool"):
            builder = ModelFactory.build_form_model

        return builder(state, tools)

    def build_llm(
        tool_choice: str = None
    ):
        params = {
            "model": LLM_MODEL,
            "temperature": 0,
            "verbose": True
        }
        if tool_choice:
            params["tool_choice"] = {
                "type": "function",
                "function": {
                    "name": tool_choice
                }
            }

        return ChatOpenAI(**params)

    def build_default_model(
        state: AgentState,
        tools: List[BaseTool] = []
    ):
        return ModelFactory.__build_model_from_state_and_prompt(
            state=state,
            prompt=DEFAULT_PROMPT_TEMPLATE,
            tools=tools
        )

    def build_form_model(
        state: AgentState,
        tools: List[BaseTool] = []
    ):

        form_tool = state.get("active_form_tool")

        information_to_collect = form_tool.get_next_field_to_collect()
        if information_to_collect:
            message = information_to_collect_prompt_template(
                form_tool, information_to_collect)
        else:
            message = ask_for_confirmation_prompt_template(form_tool)

        return ModelFactory.__build_model_from_state_and_prompt(
            state=state,
            prompt=ChatPromptTemplate(
                messages=[
                    BASE_SYSTEM_MESSAGE_PROMPT_TEMPLATE,
                    message,
                    *PROMPT_FOOTER_MESSAGES
                ]
            ),
            tools=tools
        )

    def build_error_model(
        state: AgentState,
        tools: List[BaseTool] = []
    ):
        return ModelFactory.__build_model_from_state_and_prompt(
            state=state,
            prompt=ERROR_CORRECTION_PROMPT_TEMPLATE,
            tools=tools
        )

    def __build_model_from_state_and_prompt(
        state: AgentState,
        prompt: ChatPromptTemplate,
        tools: List[BaseTool] = []
    ):
        return create_openai_tools_agent(
            ModelFactory.build_llm(state.get("tool_choice")),
            tools,
            prompt=prompt
        )


### Define the tool executor

The tool executor parses the output in a FormToolOutcome class

In [3]:
from langgraph.prebuilt.tool_executor import *

class FormToolExecutor(ToolExecutor):

    def _execute(
        self,
        tool_invocation: ToolInvocationInterface,
        agent_state: AgentState = None,
    ) -> Any:
        if tool_invocation.tool not in self.tool_map:
            return self.invalid_tool_msg_template.format(
                requested_tool_name=tool_invocation.tool,
                available_tool_names_str=", ".join(
                    [t.name for t in self.tools]),
            )
        else:
            tool = self.tool_map[tool_invocation.tool]
            output = tool.invoke(
                tool_invocation.tool_input,
                agent_state=agent_state)
            output = self._parse_tool_outcome(output, tool)
            return output

    def _parse_tool_outcome(
        self,
        output: Union[str, FormToolOutcome],
        tool: BaseTool
    ):
        if isinstance(output, str):
            return FormToolOutcome(
                state_update={},
                output=output,
                return_direct=tool.return_direct,
            )
        elif isinstance(output, FormToolOutcome):
            return output
        else:
            raise ValueError(
                f"Tool returned an invalid output: {output}. Must return a string or a FormToolOutcome.")


### Define the agent executor

The agent executor implements some logics to account for FormTools

In [4]:
import logging
import pprint
import traceback
from typing import Any, Sequence, Type

from langchain.tools import BaseTool
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.exceptions import OutputParserException
from langchain_core.language_models.chat_models import *
from langchain_core.messages import FunctionMessage
from langgraph.graph import END, StateGraph

logger = logging.getLogger(__name__)
pp = pprint.PrettyPrinter(indent=4)


class FormAgentExecutor(StateGraph):

    MAX_INTERMEDIATE_STEPS = 5

    def __init__(
        self,
        tools: Sequence[Type[Any]] = [],
        on_tool_start: callable = None,
        on_tool_end: callable = None,
    ) -> None:
        super().__init__(AgentState)

        self._on_tool_start = on_tool_start
        self._on_tool_end = on_tool_end
        self._tools = tools
        self.__build_graph()

    def __build_graph(self):

        self.add_node("agent", self.call_agent)
        self.add_node("tool", self.call_tool)

        self.add_conditional_edges(
            "agent",
            self.should_continue_after_agent,
            {
                "tool": "tool",
                "error": "agent",
                "end": END
            }
        )

        self.add_conditional_edges(
            "tool",
            self.should_continue_after_tool,
            {
                "error": "agent",
                "continue": "agent",
                "end": END
            }
        )

        self.set_entry_point("agent")
        self.app = self.compile()

    def get_tools(self, state: AgentState):
        return filter_active_tools(self._tools[:], state)

    def get_tool_by_name(self, name: str, agent_state: AgentState):
        return next((tool for tool in self.get_tools(
            agent_state) if tool.name == name), None)

    def get_tool_executor(self, state: AgentState):
        return FormToolExecutor(self.get_tools(state))

    def should_continue_after_agent(self, state: AgentState):
        if state.get("error"):
            return "error"
        elif isinstance(state.get("agent_outcome"), AgentFinish):
            return "end"
        if isinstance(state.get("agent_outcome"), list):
            return "tool"

    def should_continue_after_tool(self, state: AgentState):
        if state.get("error"):
            return "error"
        elif isinstance(state.get("tool_outcome"), FormToolOutcome) and state.get("tool_outcome").return_direct:
            return "end"
        else:
            return "continue"

    def build_model(self, state: AgentState):
        return ModelFactory.build_model(
            state=state,
            tools=self.get_tools(state)
        )

    # Define the function that calls the model
    def call_agent(self, state: AgentState):
        try:
            # Cap the number of intermediate steps in a prompt to 5
            if len(state.get("intermediate_steps")
                   ) > self.MAX_INTERMEDIATE_STEPS:
                state["intermediate_steps"] = state.get(
                    "intermediate_steps")[-self.MAX_INTERMEDIATE_STEPS:]

            agent_outcome = self.build_model(state=state).invoke(state)

            updates = {
                "agent_outcome": agent_outcome,
                "tool_choice": None,  # Reset the function call
                "tool_outcome": None,  # Reset the tool outcome
                "error": None  # Reset the error
            }
            return updates
        # TODO: if other exceptions are raised, we should handle them here
        except OutputParserException as e:
            traceback.print_exc()
            updates = {"error": str(e)}
            return updates

    def on_tool_start(self, tool: BaseTool, tool_input: dict):
        if self._on_tool_start:
            self._on_tool_start(tool, tool_input)

    def on_tool_end(self, tool: BaseTool, tool_output: Any):
        if self._on_tool_end:
            self._on_tool_end(tool, tool_output)

    def call_tool(self, state: AgentState):
        try:
            actions = state.get("agent_outcome")
            intermediate_steps = []

            for action in actions:
                tool = self.get_tool_by_name(action.tool, state)

                self.on_tool_start(tool=tool, tool_input=action.tool_input)
                tool_outcome = self.get_tool_executor(state).invoke(action)
                self.on_tool_end(tool=tool, tool_output=tool_outcome.output)

                intermediate_steps.append(
                    (
                        action,
                        FunctionMessage(
                            content=str(tool_outcome.output),
                            name=action.tool
                        )
                    )
                )

            updates = {
                **tool_outcome.state_update,
                "intermediate_steps": intermediate_steps,
                "tool_outcome": tool_outcome, # this isn't really correct with multiple tools
                "agent_outcome": None,
                "error": None
            }

        except Exception as e:
            traceback.print_exc()
            updates = {
                "intermediate_steps": [(action, FunctionMessage(
                    content=f"{type(e).__name__}: {str(e)}",
                    name=action.tool
                ))],
                "error": str(e)
            }
        finally:
            return updates

    def parse_output(self, graph_output: dict) -> str:
        """
        Parses the final state of the graph.
        Theoretically, only one between tool_outcome and agent_outcome are set.
        Returns the str to be considered the output of the graph.
        """

        state = graph_output[END]
        
        output = None
        if state.get("tool_outcome"):
            output = state.get("tool_outcome").output
        elif state.get("agent_outcome"):
            output = state.get("agent_outcome").return_values["output"]

        return output

def filter_active_tools(
    tools: Sequence[BaseTool],
    context: AgentState
):
    """
    Form tools are replaced by their activators if they are not active.
    """
    if context.get("active_form_tool"):
        # If a form_tool is active, it is the only form tool available
        base_tools = [
            tool for tool in tools if not isinstance(
                tool, FormTool)]
        tools = [
            *base_tools,
            context.get("active_form_tool"),
            FormReset(context=context)
        ]
    return tools

### Define a form tool to the the framework

We define an example tool to showcase the FormTool class. It is somewhat overly complex to show all the validation related flows.

In [5]:
from typing import Literal, Optional, Type, Any

from pydantic import BaseModel, Field, field_validator, model_validator

class OnlinePurchasePayload(BaseModel):

    allowed_provinces_: list = []

    item: Literal["watch", "shoes", "phone", "book"] = Field(
        description="Item to purchase"
    )

    ebook: Optional[bool] = Field(
        description="If true, the book will be sent as an ebook, if false it will be sent as a physical copy. Required if item is book"
    )

    email: Optional[str] = Field(
        description="Email to send the ebook"
    )

    quantity: int = Field(
        description="Quantity of items to purchase, between 1 and 10"
    )

    region: str = Field(
        description="Region to ship the item"
    )

    province: Optional[str] = Field(
        description="Province to ship the item"
    )

    address: str = Field(
        description="Address to ship the item"
    )

    @field_validator("quantity")
    def validate_quantity(cls, v):
        if v is not None:
            if v < 1 or v > 10:
                raise ValueError("Quantity must be between 1 and 10")
        return v
    
    @field_validator("region")
    def validate_region(cls, v):
        if v is not None:
            if v not in ["puglia", "sicilia", "toscana"]:
                raise ValueError("Region must be one of puglia, sicilia, toscana")
        return v
    
    @model_validator(mode="before")
    def set_allowed_provinces(cls, values: Any) -> Any:
        region = values.get("region").lower() if values.get("region") else None
        province = values.get("province").lower() if values.get("province") else None
        
        if region:
            allowed_provinces = []
            if region == "puglia":
                allowed_provinces = ["bari", "bat", "brindisi", "foggia", "lecce", "taranto"]
            if region == "sicilia":
                allowed_provinces = ["agrigento", "caltanissetta", "catania", "enna", "messina", "palermo", "ragusa", "siracusa", "trapani"]
            if region == "toscana":
                allowed_provinces = ["arezzo", "firenze", "grosseto", "livorno", "lucca", "massa-carrara", "pisa", "pistoia", "prato", "siena"]
            values.update({
                "region": region,
                "province": province,
                "allowed_provinces_": allowed_provinces
            })
        return values

    @model_validator(mode="before")
    def validate_ebook(cls, values: Any) -> Any:
        if values.get("item") == "book" and values.get("ebook") is None:
            raise ValueError("Ebook must be set for books")
        return values
    
    @model_validator(mode="after")
    def validate_province(cls, model: "OnlinePurchasePayload"):
        if model.region and model.province:
            if model.province not in model.allowed_provinces_:
                raise ValueError(f"Province must be one of {model.allowed_provinces_}")
        return model


class OnlinePurchase(FormTool):
    name = "OnlinePurchase"
    description = """Purchase an item from an online store"""
    args_schema: Type[BaseModel] = OnlinePurchasePayload


    def _run_when_complete(
        self,
        *args,
        **kwargs
    ) -> str:
        return "OK"
    
    def get_next_field_to_collect(
        self,
        **kwargs
    ) -> str:
        """
        The default implementation returns the first field that is not set.
        """
        if not self.form.item:
            return "item"
        
        if self.form.item == "book":
            if self.form.ebook == None:
                return "ebook"
            if self.form.ebook == True:
                if not self.form.email:
                    return "email"
                else:
                    return None
            
        if not self.form.quantity:
            return "quantity"
        
        if not self.form.region:
            return "region"
        
        if not self.form.province:
            return "province"
        
        if not self.form.address:
            return "address"
        
        return None

In [6]:
import os

from langchain.schema import AIMessage, HumanMessage, SystemMessage

os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_PROJECT"] = "example-project"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

graph = FormAgentExecutor(
    tools=[
        OnlinePurchase()
    ]
)

history = []
active_form_tool = None

while True:
    human_input = input("Human: ")
    if not human_input:
        break

    inputs = {
        "input": human_input,
        "chat_history": history,
        "intermediate_steps": [],
        "active_form_tool": active_form_tool
    }

    for output in graph.app.stream(inputs, config={"recursion_limit": 25}):
        for key, value in output.items():
            pass

    active_form_tool = value.get("active_form_tool")

    print(output)
    output = graph.parse_output(output)
    print(f"Human: {human_input}")
    print(f"AI: {output}")

    history = [
        *history,
        HumanMessage(content=human_input),
        AIMessage(content=output)
    ]

/home/gianfranco/.cache/pypoetry/virtualenvs/mai-assistant-TVCKXyvP-py3.10/lib/python3.10/site-packages/langchain_openai/chat_models/base.py:289: UserWarning: WARNING! tool_choice is not default parameter.
                    tool_choice was transferred to model_kwargs.
                    Please confirm that tool_choice is what you intended.
  warnings.warn(
Traceback (most recent call last):
  File "/tmp/ipykernel_250263/1213018279.py", line 135, in call_tool
    tool_outcome = self.get_tool_executor(state).invoke(action)
  File "/home/gianfranco/.cache/pypoetry/virtualenvs/mai-assistant-TVCKXyvP-py3.10/lib/python3.10/site-packages/langchain_core/runnables/base.py", line 4041, in invoke
    return self.bound.invoke(
  File "/home/gianfranco/.cache/pypoetry/virtualenvs/mai-assistant-TVCKXyvP-py3.10/lib/python3.10/site-packages/langchain_core/runnables/base.py", line 3507, in invoke
    return self._call_with_config(
  File "/home/gianfranco/.cache/pypoetry/virtualenvs/mai-assistant-TV

{'__end__': {'input': 'i want to buy a book', 'chat_history': [], 'agent_outcome': AgentFinish(return_values={'output': 'To proceed with purchasing the book, I need to know if you would like to receive it as an ebook or a physical copy. Could you please specify your preference?'}, log='To proceed with purchasing the book, I need to know if you would like to receive it as an ebook or a physical copy. Could you please specify your preference?'), 'tool_outcome': None, 'intermediate_steps': [(OpenAIToolAgentAction(tool='OnlinePurchaseStart', tool_input={}, log='\nInvoking: `OnlinePurchaseStart` with `{}`\n\n\n', message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Hgs19MMMuYQwVgh5zDpDC3oB', 'function': {'arguments': '{}', 'name': 'OnlinePurchaseStart'}, 'type': 'function'}]})], tool_call_id='call_Hgs19MMMuYQwVgh5zDpDC3oB'), FunctionMessage(content='Starting form OnlinePurchaseUpdate. If the user as already provided some information, call OnlinePurchaseUpdate.',

Traceback (most recent call last):
  File "/tmp/ipykernel_250263/1213018279.py", line 135, in call_tool
    tool_outcome = self.get_tool_executor(state).invoke(action)
  File "/home/gianfranco/.cache/pypoetry/virtualenvs/mai-assistant-TVCKXyvP-py3.10/lib/python3.10/site-packages/langchain_core/runnables/base.py", line 4041, in invoke
    return self.bound.invoke(
  File "/home/gianfranco/.cache/pypoetry/virtualenvs/mai-assistant-TVCKXyvP-py3.10/lib/python3.10/site-packages/langchain_core/runnables/base.py", line 3507, in invoke
    return self._call_with_config(
  File "/home/gianfranco/.cache/pypoetry/virtualenvs/mai-assistant-TVCKXyvP-py3.10/lib/python3.10/site-packages/langchain_core/runnables/base.py", line 1246, in _call_with_config
    context.run(
  File "/home/gianfranco/.cache/pypoetry/virtualenvs/mai-assistant-TVCKXyvP-py3.10/lib/python3.10/site-packages/langchain_core/runnables/config.py", line 326, in call_func_with_variable_args
    return func(input, **kwargs)  # type: ig

{'__end__': {'input': 'Italy', 'chat_history': [HumanMessage(content='i want to buy a book'), AIMessage(content='To proceed with purchasing the book, I need to know if you would like to receive it as an ebook or a physical copy. Could you please specify your preference?'), HumanMessage(content='physical'), AIMessage(content='Got it! Since you prefer a physical copy of the book, could you please provide the quantity of books you would like to purchase?'), HumanMessage(content='1'), AIMessage(content='Great! The book has been added to your purchase. Next, could you please provide the region where you would like the book to be shipped?')], 'agent_outcome': AgentFinish(return_values={'output': 'It seems there was an issue with the region. Please select a region from the following options: Puglia, Sicilia, Toscana.'}, log='It seems there was an issue with the region. Please select a region from the following options: Puglia, Sicilia, Toscana.'), 'tool_outcome': None, 'intermediate_steps': [

Traceback (most recent call last):
  File "/tmp/ipykernel_250263/1213018279.py", line 135, in call_tool
    tool_outcome = self.get_tool_executor(state).invoke(action)
  File "/home/gianfranco/.cache/pypoetry/virtualenvs/mai-assistant-TVCKXyvP-py3.10/lib/python3.10/site-packages/langchain_core/runnables/base.py", line 4041, in invoke
    return self.bound.invoke(
  File "/home/gianfranco/.cache/pypoetry/virtualenvs/mai-assistant-TVCKXyvP-py3.10/lib/python3.10/site-packages/langchain_core/runnables/base.py", line 3507, in invoke
    return self._call_with_config(
  File "/home/gianfranco/.cache/pypoetry/virtualenvs/mai-assistant-TVCKXyvP-py3.10/lib/python3.10/site-packages/langchain_core/runnables/base.py", line 1246, in _call_with_config
    context.run(
  File "/home/gianfranco/.cache/pypoetry/virtualenvs/mai-assistant-TVCKXyvP-py3.10/lib/python3.10/site-packages/langchain_core/runnables/config.py", line 326, in call_func_with_variable_args
    return func(input, **kwargs)  # type: ig

{'__end__': {'input': 'Roma', 'chat_history': [HumanMessage(content='i want to buy a book'), AIMessage(content='To proceed with purchasing the book, I need to know if you would like to receive it as an ebook or a physical copy. Could you please specify your preference?'), HumanMessage(content='physical'), AIMessage(content='Got it! Since you prefer a physical copy of the book, could you please provide the quantity of books you would like to purchase?'), HumanMessage(content='1'), AIMessage(content='Great! The book has been added to your purchase. Next, could you please provide the region where you would like the book to be shipped?'), HumanMessage(content='Italy'), AIMessage(content='It seems there was an issue with the region. Please select a region from the following options: Puglia, Sicilia, Toscana.'), HumanMessage(content='Puglia'), AIMessage(content='The region has been updated to Puglia. Now, could you please provide the province where you would like the book to be shipped?')], 